# SparkSession

<p>Le point d’entrée pour la programmation de Spark avec l’API Dataset et DataFrame.</p>

## point d’entrée

 <p>Un point d'entrée est l'endroit où le contrôle est transféré du système d'exploitation au programme fourni.</p><br>
<p>Pour créer une SparkSession de base, utilisez SparkSession.builder():</p>

In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

# Création de DataFrames

<p>Avec une SparkSession, les applications peuvent créer des DataFrames à partir d'un RDD existant, d'une table Hive ou de sources de données Spark.
<br>
À titre d'exemple, ce qui suit crée un DataFrame basé sur le contenu d'un fichier CSV:</p>
<h3 style="color:red;">n'oubliez pas de mettre le bon path to spark folder </h3>

In [ ]:
val df = spark.read.json("/path to spark folder/examples/src/main/resources/people.json")

// Displays the content of the DataFrame to stdout
df.show()

# Opérations DataFrame

<p>Les DataFrames fournissent un langage spécifique  pour la manipulation de données structurées en Scala, Java, Python et R.
Ces opérations sont également appelées «transformations non typées», contrairement aux «transformations typées» fournies avec des Dataset Scala / Java fortement typés.
Nous incluons ici quelques exemples de base de traitement de données structurées utilisant des ensembles de données:</p>

## printSchema

<p>printSchema imprime le schéma d'un DataFrame dans un format d'arborescence.</p>

In [ ]:
df.printSchema()

## select

La fonction select sélectionne les colonnes spécifiées et les renvoie sous la forme d'un nouveau DataFrame. Ceci est similaire à l'instruction SELECT en SQL

Sélectionnez uniquement la colonne "nom"

In [ ]:
df.select("name").show()

Sélectionnez tout le monde, mais incrémentez l'âge de 1

In [ ]:
df.select(df("name"), df("age") + 1).show()

## filter

Le filtre retourne les lignes d'un DataFrame correspondant à la condition donnée.

Sélectionnez les personnes de plus de 21 ans

In [ ]:
df.filter($"age" > 21).show()

## groupBy

La fonction groupBy regroupe votre DataFrame sur la ou les colonnes spécifiées afin que vous puissiez y exécuter des agrégations, un peu comme votre instruction SQL GROUP BY. La fonction groupBy génère un objet GroupedData qui peut ensuite être transmis aux fonctions d'agrégation.

Compter les gens par âge

In [ ]:
df.groupBy("age").count().show()

# Exécution de requêtes SQL par programme

## sql

La fonction sql sur SparkSession permet aux applications d'exécuter des requêtes SQL par programme et renvoie le résultat sous la forme d'un DataFrame.

## createOrReplaceTempView

Crée une nouvelle vue temporaire à l'aide de SparkDataFrame dans la session Spark. Si une vue temporaire portant le même nom existe déjà, la remplace.

In [ ]:
df.createOrReplaceTempView("people")
val sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

# Global Temporary View

Les vues temporaires dans Spark SQL ont une portée de session et disparaissent si la session qui les crée se termine. Si vous souhaitez avoir une vue temporaire partagée entre toutes les sessions et rester en vie jusqu'à la fin de l'application Spark, vous pouvez créer une vue temporaire globale. La vue temporaire globale est liée à une base de données préservée par le système, global_temp, et nous devons utiliser le nom qualifié pour la référencer, par exemple. SELECT * FROM global_temp.view1.

In [ ]:
// Enregistrer le DataFrame en tant que vue temporaire globale
df.createGlobalTempView("people")

// La vue temporaire globale est liée à une base de données préservée du système, `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()


// La vue temporaire globale est intersession
spark.newSession().sql("SELECT * FROM global_temp.people").show()


# Création datasets

Les ensembles de données sont similaires aux RDD. Cependant, au lieu d'utiliser la sérialisation Java, ils utilisent un encodeur spécialisé pour sérialiser les objets à traiter ou à transmettre.

Création d'une class case Person

In [ ]:
case class Person(name: String, age: Long)

encodage  dune seq  d'objets de type Person  en Dataset

In [ ]:
val caseClassDS = Seq(Person("Andy", 32)).toDS()
caseClassDS.show()

Les encodeurs pour les types les plus courants sont automatiquement fournis en important spark.implicits._

In [ ]:
val primitiveDS = Seq(1, 2, 3).toDS()
primitiveDS.show()

Les DataFrames peuvent être convertis en un jeu de données en fournissant une classe. La cartographie sera faite par nom

In [ ]:
val peopleDS = df.as[Person]
peopleDS.show()